# Data Processing - PART 2
## Encoding
Read final_data.csv<br>
Based on domain understanding encode categorical data appropriately<br>
Get data ready for modelling<br>
Write data to model_data.csv

In [1]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy


import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV

%matplotlib inline
%pylab inline 
%config IPCompleter.greedy=True
%config InlineBackend.figure_format = 'svg'
sns.set_style("white")

Populating the interactive namespace from numpy and matplotlib


In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 30)

### 1. Read Data
from final_data.csv already cleaned data

In [3]:
cars = pd.read_csv('../data/final_data.csv')
cars = cars.drop(columns="Unnamed: 0")
cars.head()

,stock_n,year,brand,model,price,mileage,interior_color,exterior_color,drive_type,engine_size,num_cylinders,hp,comp_ratio,cam_shaft,engine_type,bore,stroke,values_per_cylinder,wheel_base,length,width,height,curb_weight,leg_room,head_room,seating_cap,cargo_cap,city_mpg,hw_mpg
0,16568245,2016,Acura,ILX,19998.0,31000,Black,Black,2,2.4,4,201,12:1,Variable Timing,Gas,3.43,3.90,4.0,105.1,181.9,70.0,55.0,2970,42.3,38.0,5,12.0,25,35
1,16238238,2013,Audi,Q5 Premium Plus,26998.0,48000,Black,Gray,4,3.0,6,272,10:1,Dual Overhead Cam,Turbo Gas,3.33,3.50,0.0,110.5,182.6,74.0,65.0,4345,41.0,38.1,5,57.0,18,26
2,15411470,2013,Audi,RS5,41998.0,37000,Black,White,4,4.2,8,450,13:1,Dual Overhead Cam,Gas,3.33,3.65,0.0,108.3,183.0,73.0,53.0,3924,41.3,39.1,4,12.0,16,23
3,15660515,2011,Audi,TTS Prestige,22998.0,41000,Black,Orange,4,2.0,4,265,9.8:1,Dual Overhead Cam,Turbo Gas,3.25,3.65,4.0,97.2,165.3,72.0,53.0,3241,41.1,37.7,4,13.0,21,29
4,16238171,2015,BMW,320 XI,20998.0,14000,Tan,Red,4,2.0,4,180,11:1,Variable Timing DOHC,Turbo Gas,3.31,3.55,4.0,110.6,182.1,71.0,56.0,3450,42.0,40.3,5,17.0,23,35


In [4]:
cars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1781 entries, 0 to 1780
Data columns (total 29 columns):
stock_n                1781 non-null int64
year                   1781 non-null int64
brand                  1781 non-null object
model                  1781 non-null object
price                  1781 non-null float64
mileage                1781 non-null int64
interior_color         1781 non-null object
exterior_color         1781 non-null object
drive_type             1781 non-null int64
engine_size            1781 non-null float64
num_cylinders          1781 non-null int64
hp                     1781 non-null int64
comp_ratio             1781 non-null object
cam_shaft              1781 non-null object
engine_type            1781 non-null object
bore                   1781 non-null float64
stroke                 1781 non-null float64
values_per_cylinder    1781 non-null float64
wheel_base             1781 non-null float64
length                 1781 non-null float64
width       

### 2. Seperate out features

In [5]:
cars.columns
# our target
cars_y=['price']
# numeric continous variables
cars_num=['mileage','engine_size','hp','bore','stroke','wheel_base', 'length', 'width','height',\
          'curb_weight', 'leg_room', 'head_room', 'seating_cap','cargo_cap', 'city_mpg', 'hw_mpg', 'comp_ratio']
# ordered categorical
cars_oc = ['year','drive_type','num_cylinders','values_per_cylinder']

# needs one hot encoding
cars_cat = ['brand','interior_color', 'exterior_color','engine_type']
          
# variables not needed for modelling 
cars_other = ['stock_n','model', 'cam_shaft']

### 3. Encode brands as luxury or not luxury

In [6]:
# encoding for luxury brands 1 normal = 0
lux_car=['Acura', 'Audi', 'BMW', 'Infiniti', 'Jaguar','Land','Lexus',\
         'Lincoln', 'Mercedes-Benz', 'Chrysler','Volvo','Cadillac', 'Mini', 'Buick','Porsche']
cars['luxury']=0
cars.loc[cars['brand'].isin(lux_car),'luxury']=1
cars.head()

,stock_n,year,brand,model,price,mileage,interior_color,exterior_color,drive_type,engine_size,num_cylinders,hp,comp_ratio,cam_shaft,engine_type,bore,stroke,values_per_cylinder,wheel_base,length,width,height,curb_weight,leg_room,head_room,seating_cap,cargo_cap,city_mpg,hw_mpg,luxury
0,16568245,2016,Acura,ILX,19998.0,31000,Black,Black,2,2.4,4,201,12:1,Variable Timing,Gas,3.43,3.90,4.0,105.1,181.9,70.0,55.0,2970,42.3,38.0,5,12.0,25,35,1
1,16238238,2013,Audi,Q5 Premium Plus,26998.0,48000,Black,Gray,4,3.0,6,272,10:1,Dual Overhead Cam,Turbo Gas,3.33,3.50,0.0,110.5,182.6,74.0,65.0,4345,41.0,38.1,5,57.0,18,26,1
2,15411470,2013,Audi,RS5,41998.0,37000,Black,White,4,4.2,8,450,13:1,Dual Overhead Cam,Gas,3.33,3.65,0.0,108.3,183.0,73.0,53.0,3924,41.3,39.1,4,12.0,16,23,1
3,15660515,2011,Audi,TTS Prestige,22998.0,41000,Black,Orange,4,2.0,4,265,9.8:1,Dual Overhead Cam,Turbo Gas,3.25,3.65,4.0,97.2,165.3,72.0,53.0,3241,41.1,37.7,4,13.0,21,29,1
4,16238171,2015,BMW,320 XI,20998.0,14000,Tan,Red,4,2.0,4,180,11:1,Variable Timing DOHC,Turbo Gas,3.31,3.55,4.0,110.6,182.1,71.0,56.0,3450,42.0,40.3,5,17.0,23,35,1


### 4. Encode for interior color
Almost all are black as this is often as standard <br>
Black or not Black

In [7]:
cars['interior_color'].value_counts()

Black       1177
Gray         366
Tan          187
Brown         29
Cream         10
Red            8
Burgundy       2
White          1
Taupe          1
Name: interior_color, dtype: int64

In [8]:
cars['black_interior'] = 0
cars.loc[cars['interior_color']=='Black','black_interior']=1
cars.head()

,stock_n,year,brand,model,price,mileage,interior_color,exterior_color,drive_type,engine_size,num_cylinders,hp,comp_ratio,cam_shaft,engine_type,bore,stroke,values_per_cylinder,wheel_base,length,width,height,curb_weight,leg_room,head_room,seating_cap,cargo_cap,city_mpg,hw_mpg,luxury,black_interior
0,16568245,2016,Acura,ILX,19998.0,31000,Black,Black,2,2.4,4,201,12:1,Variable Timing,Gas,3.43,3.90,4.0,105.1,181.9,70.0,55.0,2970,42.3,38.0,5,12.0,25,35,1,1
1,16238238,2013,Audi,Q5 Premium Plus,26998.0,48000,Black,Gray,4,3.0,6,272,10:1,Dual Overhead Cam,Turbo Gas,3.33,3.50,0.0,110.5,182.6,74.0,65.0,4345,41.0,38.1,5,57.0,18,26,1,1
2,15411470,2013,Audi,RS5,41998.0,37000,Black,White,4,4.2,8,450,13:1,Dual Overhead Cam,Gas,3.33,3.65,0.0,108.3,183.0,73.0,53.0,3924,41.3,39.1,4,12.0,16,23,1,1
3,15660515,2011,Audi,TTS Prestige,22998.0,41000,Black,Orange,4,2.0,4,265,9.8:1,Dual Overhead Cam,Turbo Gas,3.25,3.65,4.0,97.2,165.3,72.0,53.0,3241,41.1,37.7,4,13.0,21,29,1,1
4,16238171,2015,BMW,320 XI,20998.0,14000,Tan,Red,4,2.0,4,180,11:1,Variable Timing DOHC,Turbo Gas,3.31,3.55,4.0,110.6,182.1,71.0,56.0,3450,42.0,40.3,5,17.0,23,35,1,0


### 5. Encode for exterior color
Classify the category in dark, light or prime <br>
dark,light, prime

In [9]:
# encoding exterior colors

cars['exterior_color'].value_counts()
dark = ['Black','Gray','Brown']
light = ['White','Pearl','Silver']
prime =['Blue','Red','Burgundy','Green','Gold','Orange','Tan','Purple','Yellow','Maroon']

cars['dark_exterior'] = 0
cars.loc[cars['exterior_color'].isin(dark),'dark_exterior']=1

cars['light_exterior'] = 0
cars.loc[cars['exterior_color'].isin(light),'light_exterior']=1

cars['prime_exterior'] = 0
cars.loc[cars['exterior_color'].isin(prime),'prime_exterior']=1


cars.head()

,stock_n,year,brand,model,price,mileage,interior_color,exterior_color,drive_type,engine_size,num_cylinders,hp,comp_ratio,cam_shaft,engine_type,bore,stroke,values_per_cylinder,wheel_base,length,width,height,curb_weight,leg_room,head_room,seating_cap,cargo_cap,city_mpg,hw_mpg,luxury,black_interior,dark_exterior,light_exterior,prime_exterior
0,16568245,2016,Acura,ILX,19998.0,31000,Black,Black,2,2.4,4,201,12:1,Variable Timing,Gas,3.43,3.90,4.0,105.1,181.9,70.0,55.0,2970,42.3,38.0,5,12.0,25,35,1,1,1,0,0
1,16238238,2013,Audi,Q5 Premium Plus,26998.0,48000,Black,Gray,4,3.0,6,272,10:1,Dual Overhead Cam,Turbo Gas,3.33,3.50,0.0,110.5,182.6,74.0,65.0,4345,41.0,38.1,5,57.0,18,26,1,1,1,0,0
2,15411470,2013,Audi,RS5,41998.0,37000,Black,White,4,4.2,8,450,13:1,Dual Overhead Cam,Gas,3.33,3.65,0.0,108.3,183.0,73.0,53.0,3924,41.3,39.1,4,12.0,16,23,1,1,0,1,0
3,15660515,2011,Audi,TTS Prestige,22998.0,41000,Black,Orange,4,2.0,4,265,9.8:1,Dual Overhead Cam,Turbo Gas,3.25,3.65,4.0,97.2,165.3,72.0,53.0,3241,41.1,37.7,4,13.0,21,29,1,1,0,0,1
4,16238171,2015,BMW,320 XI,20998.0,14000,Tan,Red,4,2.0,4,180,11:1,Variable Timing DOHC,Turbo Gas,3.31,3.55,4.0,110.6,182.1,71.0,56.0,3450,42.0,40.3,5,17.0,23,35,1,0,0,0,1


### 6. Encode for Engine_types
group engine_types based on there fuel use  <br>
normal,turbo, alternate

In [10]:
cars['engine_type'].value_counts()

Gas             1189
Turbo Gas        338
Ethanol/Gas      133
Gas/Electric      90
Supercharged      17
Turbo Diesel      13
gas                1
Name: engine_type, dtype: int64

In [11]:
# encoding engine_type 
# consider supercharged to be in the turbo category as they are very similat
normal =['Gas']
turbo= ['Turbo Gas','Turbo Diesel','Supercharged']
alternate=['Ethanol/Gas','Gas/Electric']

cars['normal_engine'] = 0
cars.loc[cars['engine_type'].isin(normal),'normal_engine']=1

cars['turbo_engine'] = 0
cars.loc[cars['engine_type'].isin(turbo),'turbo_engine']= 1

cars['alternate_engine'] = 0
cars.loc[cars['engine_type'].isin(alternate),'alternate_engine']= 1


cars.head()

,stock_n,year,brand,model,price,mileage,interior_color,exterior_color,drive_type,engine_size,num_cylinders,hp,comp_ratio,cam_shaft,engine_type,bore,stroke,values_per_cylinder,wheel_base,length,width,height,curb_weight,leg_room,head_room,seating_cap,cargo_cap,city_mpg,hw_mpg,luxury,black_interior,dark_exterior,light_exterior,prime_exterior,normal_engine,turbo_engine,alternate_engine
0,16568245,2016,Acura,ILX,19998.0,31000,Black,Black,2,2.4,4,201,12:1,Variable Timing,Gas,3.43,3.90,4.0,105.1,181.9,70.0,55.0,2970,42.3,38.0,5,12.0,25,35,1,1,1,0,0,1,0,0
1,16238238,2013,Audi,Q5 Premium Plus,26998.0,48000,Black,Gray,4,3.0,6,272,10:1,Dual Overhead Cam,Turbo Gas,3.33,3.50,0.0,110.5,182.6,74.0,65.0,4345,41.0,38.1,5,57.0,18,26,1,1,1,0,0,0,1,0
2,15411470,2013,Audi,RS5,41998.0,37000,Black,White,4,4.2,8,450,13:1,Dual Overhead Cam,Gas,3.33,3.65,0.0,108.3,183.0,73.0,53.0,3924,41.3,39.1,4,12.0,16,23,1,1,0,1,0,1,0,0
3,15660515,2011,Audi,TTS Prestige,22998.0,41000,Black,Orange,4,2.0,4,265,9.8:1,Dual Overhead Cam,Turbo Gas,3.25,3.65,4.0,97.2,165.3,72.0,53.0,3241,41.1,37.7,4,13.0,21,29,1,1,0,0,1,0,1,0
4,16238171,2015,BMW,320 XI,20998.0,14000,Tan,Red,4,2.0,4,180,11:1,Variable Timing DOHC,Turbo Gas,3.31,3.55,4.0,110.6,182.1,71.0,56.0,3450,42.0,40.3,5,17.0,23,35,1,0,0,0,1,0,1,0


### 7. Drop columns
drop columns that are not relevant for modelling .. stockno etc.<br>
drop the categorical columns that have now be encoded<br>
refer section 2 of notebook<br>
fix comp_ratio so that it is float - missed during data cleaning

In [12]:
cars = cars.drop(columns = cars_cat)
cars = cars.drop(columns = cars_other)

In [13]:
#clean comp_ratio
cars['comp_ratio'] = cars['comp_ratio'].str.split(':',n=1,expand=True)
cars['comp_ratio'] = cars['comp_ratio'].str.strip().astype('float')

### 8. Check Dataframe

In [14]:
cars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1781 entries, 0 to 1780
Data columns (total 30 columns):
year                   1781 non-null int64
price                  1781 non-null float64
mileage                1781 non-null int64
drive_type             1781 non-null int64
engine_size            1781 non-null float64
num_cylinders          1781 non-null int64
hp                     1781 non-null int64
comp_ratio             1781 non-null float64
bore                   1781 non-null float64
stroke                 1781 non-null float64
values_per_cylinder    1781 non-null float64
wheel_base             1781 non-null float64
length                 1781 non-null float64
width                  1635 non-null float64
height                 1779 non-null float64
curb_weight            1781 non-null int64
leg_room               1764 non-null float64
head_room              1774 non-null float64
seating_cap            1781 non-null int64
cargo_cap              1673 non-null float64
city_mp

In [15]:
cars.columns

Index(['year', 'price', 'mileage', 'drive_type', 'engine_size',
       'num_cylinders', 'hp', 'comp_ratio', 'bore', 'stroke',
       'values_per_cylinder', 'wheel_base', 'length', 'width', 'height',
       'curb_weight', 'leg_room', 'head_room', 'seating_cap', 'cargo_cap',
       'city_mpg', 'hw_mpg', 'luxury', 'black_interior', 'dark_exterior',
       'light_exterior', 'prime_exterior', 'normal_engine', 'turbo_engine',
       'alternate_engine'],
      dtype='object')

In [16]:
# reorder price to be first column
cars = cars[['price','year', 'mileage', 'drive_type', 'engine_size',
       'num_cylinders', 'hp', 'comp_ratio', 'bore', 'stroke',
       'values_per_cylinder', 'wheel_base', 'length', 'width', 'height',
       'curb_weight', 'leg_room', 'head_room', 'seating_cap', 'cargo_cap',
       'city_mpg', 'hw_mpg', 'luxury', 'black_interior', 'dark_exterior',
       'light_exterior', 'prime_exterior', 'normal_engine', 'turbo_engine',
       'alternate_engine']]

In [17]:
cars.head()

,price,year,mileage,drive_type,engine_size,num_cylinders,hp,comp_ratio,bore,stroke,values_per_cylinder,wheel_base,length,width,height,curb_weight,leg_room,head_room,seating_cap,cargo_cap,city_mpg,hw_mpg,luxury,black_interior,dark_exterior,light_exterior,prime_exterior,normal_engine,turbo_engine,alternate_engine
0,19998.0,2016,31000,2,2.4,4,201,12.0,3.43,3.90,4.0,105.1,181.9,70.0,55.0,2970,42.3,38.0,5,12.0,25,35,1,1,1,0,0,1,0,0
1,26998.0,2013,48000,4,3.0,6,272,10.0,3.33,3.50,0.0,110.5,182.6,74.0,65.0,4345,41.0,38.1,5,57.0,18,26,1,1,1,0,0,0,1,0
2,41998.0,2013,37000,4,4.2,8,450,13.0,3.33,3.65,0.0,108.3,183.0,73.0,53.0,3924,41.3,39.1,4,12.0,16,23,1,1,0,1,0,1,0,0
3,22998.0,2011,41000,4,2.0,4,265,9.8,3.25,3.65,4.0,97.2,165.3,72.0,53.0,3241,41.1,37.7,4,13.0,21,29,1,1,0,0,1,0,1,0
4,20998.0,2015,14000,4,2.0,4,180,11.0,3.31,3.55,4.0,110.6,182.1,71.0,56.0,3450,42.0,40.3,5,17.0,23,35,1,0,0,0,1,0,1,0


### 9. Write to file
write to datafile - model_data.csv

In [18]:
cars.to_csv('../data/model_data.csv')

### 10. Check 

In [19]:
df_model = pd.read_csv('../data/model_data.csv')

In [20]:
df_model.tail()

,Unnamed: 0,price,year,mileage,drive_type,engine_size,num_cylinders,hp,comp_ratio,bore,stroke,values_per_cylinder,wheel_base,length,width,height,curb_weight,leg_room,head_room,seating_cap,cargo_cap,city_mpg,hw_mpg,luxury,black_interior,dark_exterior,light_exterior,prime_exterior,normal_engine,turbo_engine,alternate_engine
1776,1776,39998.0,2016,16000,2,3.0,6,329,11.0,3.46,3.23,4.0,108.7,185.0,70.0,55.0,4101,42.0,36.8,4,12.0,20,29,1,0,0,1,0,0,1,0
1777,1777,23599.0,2013,53000,4,2.1,4,200,16.0,3.27,3.90,4.0,108.5,178.3,NaN,66.0,4246,41.4,39.0,5,55.0,24,33,1,1,0,1,0,0,1,0
1778,1778,49998.0,2014,37000,4,5.5,8,518,10.0,3.86,3.58,4.0,114.8,189.1,75.0,70.0,5238,40.3,38.9,5,36.0,13,17,1,1,0,1,0,0,1,0
1779,1779,38998.0,2013,45000,4,4.6,8,429,11.0,3.70,3.39,4.0,124.6,206.5,NaN,58.0,4861,41.9,37.8,5,16.0,15,24,1,1,1,0,0,0,1,0
1780,1780,38998.0,2010,54000,2,6.3,8,518,11.0,4.02,3.72,4.0,124.6,206.5,83.0,58.0,4741,41.9,37.8,5,16.0,11,18,1,1,1,0,0,1,0,0


### next-> explore_correlations